In [1]:
from tqdm.auto import tqdm 
import pandas as pd
import numpy as np
import xarray as xr
import netCDF4 as nf
from netCDF4 import Dataset
%matplotlib inline
import glob
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [2]:
datapath="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/Data/ECMWF/ERA5_25kmx3hr/"
path="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/"
output="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/"
target="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/targets/"

In [3]:
tracklist = sorted(glob.glob(path+'tracks_nio/*2019*'))

In [4]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20190101.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20190426.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20190609.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20191022.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20191028.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20191103.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20191130.csv']

In [5]:
dm1=xr.open_mfdataset([datapath+'/vor/vor_2019.nc',datapath+'/rhum/rhum_2019.nc'])

In [6]:
def largearea_withpres(dataset=None,invar=None,indices=None,tc_irad=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),len(dataset.plev.data),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        for ip in range(len(dm1.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out  

In [7]:
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out
###############################read year from data and track ####################################

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/svars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

####change year below##########################
dm2,tracklist,era5_date,era5_hour = readyear(2019) 

TCsl_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['u10','v10','2mdewtmp','2mtmp','conv_ppt','tot_cld_ice',\
           'tot_cldwtr','tot_cld_rain','vi_div_cld_froz_wtr','vi_div_cld_liq_wtr','vi_div_gpot_flux',\
           'vi_div_ke_flux','vi_div_mass_flux','vi_div_moisture_flux','vi_div_olr_flux','vi_div_tot_enrgy_flux',\
           'vi_ke','vi_pe_inte','vi_pe_ie_latentenrgy','vi_temp','vi_olr','vi_tot_enrgy','vi_moisture_div']
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCsl_ts.append(tsdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [8]:
##########radiation variables############
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out
###############################read year from data and track ####################################

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/radvars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2019)

TCrl_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['cape','inst_10m_wnd_gst','inst_moisture_flux','inst_ssh_flux','surfmean_swr_flux','surfmean_lhf',\
           'surfmean_lwr_flux','surfmean_shf','dwnwrdmean_swr_flux','topmean_lwr_flux','topmean_swr_flux',\
           'vimean_moisture_div','surf_lhf','surf_shf','tot_suprcool_liqwtr','tot_wtr_vpr']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCrl_ts.append(tsdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [9]:
##########precip, mslp, sst variables############
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def smallarea(dataset=None,invar=None,indices=None,tc_irad=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    LATN,LATS,LONE,LONW = tc_irad[0,:]
    testsmall = ds['variable'][0,:,:].sel(lat=slice(LATS,LATN),lon=slice(LONE,LONW))
    if testsmall.shape[0]<testsmall.shape[1]:
        rgspt = int(testsmall.shape[0])
    else:
        rgspt = int(testsmall.shape[1])
    var_out=np.zeros((len(indices),rgspt,rgspt))
    del testsmall
    
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_irad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:rgspt,0:rgspt]
    return var_out
###############################read year from data and track ####################################

def readyear(year=None):
    dm2 = xr.open_mfdataset([datapath+'/prate/prates_2019.nc',datapath+'/mslp/mslp_2019.nc',datapath+'/sst/sst_2019.nc'])
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2019)

TCpr_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    ################################################################################################
    
    smallsvarout = [smallarea(dm2,mysvar[i],indices_store,tc_irad) for i in (range(len(mysvar)))]
    
    svarname = ['mslp','conv_rrate','ls_rrate','mn_conv_prate','mn_ls_prate','mn_tot_prate','sst']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    ##################################################################################################
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...].flatten() for i in range(len(indices_store))]
        tsdict[svarname[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCpr_ts.append(tsdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [10]:
#For vars with 21 vertical levels vorticity humidity file##################
TCp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2 
    
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['vort','rhum']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCp_ts.append(ts_pdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [11]:
dm1=xr.open_dataset(datapath+'/gpot/gpot_2019.nc')

In [12]:
#geopotential
TCgp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2 
    
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['gpot']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCgp_ts.append(ts_pdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [13]:
dm1=xr.open_dataset(datapath+'/vvel/w_wnd_2019.nc')

In [14]:
#Vertical velocity
TCvv_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2 
    
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store,tc_irad) for i in (range(len(mypvar)))]    
    pvarname = ['vvel']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...].flatten() for i in range(len(indices_store))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCvv_ts.append(ts_pdict)

  0%|          | 0/7 [00:00<?, ?it/s]

In [15]:
preslv = [str(int(obj)) for obj in dm1.plev.data/100]

In [16]:
dx=xr.open_dataset(datapath+'/tlevs/tlevs_2019.nc')

In [17]:
plev=[str(int(obj)) for obj in dx.plev.data/100]

In [18]:
maindf_pabuk=pd.read_csv(target+'2019_w10max_nio_pabuk.csv',delimiter=r",")
maindf_pabuk.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_pabuk=maindf_pabuk.drop('a', axis=1)

extra=pd.read_csv(output+'nio/inder_2019_nio_pabuk.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)


In [19]:
rlev_pabuk=pd.DataFrame.from_dict(TCrl_ts[0])
prlev_pabuk=pd.DataFrame.from_dict(TCpr_ts[0])
slev_pabuk=pd.DataFrame.from_dict(TCsl_ts[0])
prlev_pabuk['mslp']=prlev_pabuk['mslp']/100

In [20]:
ts1_pabuk=pd.concat([maindf_pabuk,extra,slev_pabuk,rlev_pabuk,prlev_pabuk], axis=1, join='inner')

In [21]:
#storm1
tempvortdict = {'vort_'+plev[i]:TCp_ts[0]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[0]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[0]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[0]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

In [22]:
plev_pabuk=pd.DataFrame.from_dict(alldict)

In [23]:
ts_pabuk=pd.concat([ts1_pabuk,plev_pabuk], axis=1, join='inner')

In [25]:
ts_pabuk.to_csv(output+'inner/inner_nio_all_pabuk.csv')

In [26]:
#storms 2
maindf_fani=pd.read_csv(target+'2019_w10max_nio_fani.csv',delimiter=r",")
maindf_fani.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_fani=maindf_fani.drop('a', axis=1)

extra=pd.read_csv(output+'nio/inder_2019_nio_fani.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_fani=pd.DataFrame.from_dict(TCrl_ts[1])
prlev_fani=pd.DataFrame.from_dict(TCpr_ts[1])
slev_fani=pd.DataFrame.from_dict(TCsl_ts[1])
prlev_fani['mslp']=prlev_fani['mslp']/100

ts1_fani=pd.concat([maindf_fani,extra,slev_fani,rlev_fani,prlev_fani], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[1]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[1]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[1]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[1]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_fani=pd.DataFrame.from_dict(alldict)

ts_fani=pd.concat([ts1_fani,plev_fani], axis=1, join='inner')

ts_fani

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,11.400653,0.000031,2.972776e-05,2.790465e-05,1.523896e-05,-4.722499e-06,-1.854154e-05,337.450256,353.566132,351.941254,...,-0.499886,-0.609107,-0.651503,-0.503074,-0.333082,-0.288794,-0.205780,-0.092661,-0.023008,0.010179
1,11.019793,0.000025,1.319229e-05,2.150614e-05,1.045938e-05,-7.808180e-07,-7.757256e-06,337.550262,353.347137,351.478638,...,-0.347735,-0.432707,-0.479525,-0.408224,-0.344915,-0.335015,-0.218122,-0.097013,-0.021748,0.013618
2,10.929685,0.000032,7.849214e-06,1.356332e-05,1.016111e-05,1.522230e-06,-6.381979e-06,338.036530,353.720459,351.772217,...,-0.411079,-0.458588,-0.539495,-0.488679,-0.467668,-0.499656,-0.349854,-0.197745,-0.090669,-0.037715
3,11.506803,0.000036,3.133804e-05,2.152325e-05,9.744440e-06,6.086630e-06,2.497742e-06,337.763184,353.997375,352.516296,...,-0.531584,-0.603556,-0.683069,-0.717950,-0.765182,-0.777483,-0.497163,-0.261118,-0.100327,-0.023367
4,11.302110,0.000035,2.847285e-05,2.720746e-05,2.711894e-05,1.331085e-05,-9.333564e-06,337.598145,353.925232,352.077393,...,-0.435761,-0.572312,-0.798389,-0.814232,-0.731106,-0.714393,-0.468975,-0.234261,-0.063110,0.018509
5,12.590420,0.000043,3.265102e-05,3.228831e-05,2.692533e-05,-6.604095e-06,-1.479017e-05,338.028076,353.260498,352.389771,...,-0.586859,-0.737353,-0.869253,-0.735068,-0.659439,-0.704720,-0.495138,-0.270550,-0.093366,-0.007476
6,11.663575,0.000048,4.096503e-05,2.946241e-05,1.541522e-05,-1.261303e-05,-1.330100e-05,337.172180,354.180725,353.050598,...,-0.737019,-0.848428,-0.861888,-0.728613,-0.668651,-0.751865,-0.587763,-0.355151,-0.149531,-0.051499
7,13.626658,0.000059,4.092578e-05,2.383220e-05,7.197282e-06,-1.688728e-05,-1.641414e-05,337.145264,354.651611,352.954834,...,-0.850791,-0.930682,-0.840139,-0.681557,-0.570093,-0.630365,-0.529255,-0.321694,-0.133778,-0.045174
8,13.791686,0.000084,4.927937e-05,2.495812e-05,1.167765e-05,-1.005259e-05,-1.680227e-05,335.905579,354.754944,353.461182,...,-1.105198,-1.192042,-1.190598,-1.028391,-0.888367,-0.791352,-0.509164,-0.303210,-0.109058,-0.012432
9,14.097275,0.000075,4.417671e-05,2.798810e-05,1.192389e-05,-4.870115e-06,-6.731329e-06,337.402527,354.687103,353.864929,...,-0.970445,-1.066168,-1.092827,-1.030167,-0.965616,-0.801381,-0.423421,-0.245062,-0.088840,-0.011613


In [27]:
ts_fani.to_csv(output+'inner/inner_nio_all_fani.csv')

In [28]:
#storms 3
maindf_vayu=pd.read_csv(target+'2019_w10max_nio_vayu.csv',delimiter=r",")
maindf_vayu.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_vayu=maindf_vayu.drop('a', axis=1)

extra=pd.read_csv(output+'nio/inder_2019_nio_vayu.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_vayu=pd.DataFrame.from_dict(TCrl_ts[2])
prlev_vayu=pd.DataFrame.from_dict(TCpr_ts[2])
slev_vayu=pd.DataFrame.from_dict(TCsl_ts[2])
prlev_vayu['mslp']=prlev_vayu['mslp']/100

ts1_vayu=pd.concat([maindf_vayu,extra,slev_vayu,rlev_vayu,prlev_vayu], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[2]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[2]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[2]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[2]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_vayu=pd.DataFrame.from_dict(alldict)

ts_vayu=pd.concat([ts1_vayu,plev_vayu], axis=1, join='inner')

ts_vayu

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,13.432127,0.000037,3.033961e-05,0.000022,0.000021,1.033537e-05,-9.683152e-06,336.310120,354.170898,351.824097,...,-0.449404,-0.572090,-0.750404,-0.749943,-0.697266,-0.663045,-0.380208,-0.222608,-0.089518,-0.023504
1,14.378722,0.000057,5.648138e-05,0.000029,0.000017,-4.290808e-06,-1.818451e-05,333.951965,355.364471,352.770569,...,-0.853569,-0.972868,-1.015555,-0.899487,-0.793825,-0.825112,-0.553271,-0.312989,-0.104453,-0.002042
2,15.848115,0.000053,5.462169e-05,0.000026,0.000002,-1.273937e-05,-1.564485e-05,337.823547,356.410461,353.588165,...,-0.855945,-0.920815,-0.830833,-0.699368,-0.630118,-0.665415,-0.543692,-0.312471,-0.125721,-0.033430
3,16.375698,0.000064,3.901983e-05,0.000005,-0.000020,-2.690667e-05,-9.921511e-06,339.066284,356.989227,353.763580,...,-0.770686,-0.727422,-0.450110,-0.253179,-0.262685,-0.377176,-0.417525,-0.239383,-0.107566,-0.040173
4,14.064686,0.000055,4.183283e-05,0.000012,-0.000021,-2.480903e-05,7.740191e-07,338.007965,356.537231,353.998108,...,-0.662478,-0.620640,-0.368954,-0.246914,-0.347504,-0.498668,-0.544141,-0.319567,-0.110705,-0.010929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,11.845954,0.000002,2.820352e-06,0.000003,-0.000004,-4.722464e-06,1.586464e-06,316.065125,357.756592,353.019073,...,-0.019360,-0.012618,-0.015879,0.019693,-0.054193,-0.019276,-0.012094,-0.101584,-0.061818,-0.015912
71,12.091147,-0.000002,9.092987e-07,0.000004,-0.000005,-2.378558e-06,6.535027e-06,314.548340,357.496460,352.857971,...,-0.016006,-0.014642,-0.004347,-0.025156,-0.096189,-0.074815,-0.072983,-0.125170,-0.062085,-0.011294
72,11.416389,-0.000010,-4.606551e-06,0.000005,-0.000003,-1.952885e-06,3.420026e-06,312.680145,357.473114,353.000244,...,0.043392,0.025939,0.062410,0.031309,0.039732,-0.005217,-0.087042,-0.105604,-0.043265,-0.003667
73,11.119370,-0.000007,-6.297744e-06,0.000003,0.000001,9.115012e-07,2.443341e-06,310.716858,357.702118,353.444580,...,0.003962,-0.018911,0.004150,-0.034240,0.020796,-0.062466,-0.126332,-0.097484,-0.038214,-0.003403


In [29]:
ts_vayu.to_csv(output+'inner/inner_nio_all_vayu.csv')

In [30]:
#storms 4
maindf_kyaar=pd.read_csv(target+'2019_w10max_nio_kyaar.csv',delimiter=r",")
maindf_kyaar.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_kyaar=maindf_kyaar.drop('a', axis=1)

extra=pd.read_csv(output+'nio/inder_2019_nio_kyaar.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_kyaar=pd.DataFrame.from_dict(TCrl_ts[3])
prlev_kyaar=pd.DataFrame.from_dict(TCpr_ts[3])
slev_kyaar=pd.DataFrame.from_dict(TCsl_ts[3])
prlev_kyaar['mslp']=prlev_kyaar['mslp']/100

ts1_kyaar=pd.concat([maindf_kyaar,extra,slev_kyaar,rlev_kyaar,prlev_kyaar], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[3]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[3]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[3]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[3]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_kyaar=pd.DataFrame.from_dict(alldict)

ts_kyaar=pd.concat([ts1_kyaar,plev_kyaar], axis=1, join='inner')

ts_kyaar

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,12.210130,3.294097e-05,0.000011,-4.351196e-06,-8.201038e-06,-1.458761e-05,-0.000004,335.664368,351.300537,349.701172,...,-0.216473,-0.177923,-0.065963,0.034910,0.020925,-0.043584,-0.084535,-0.070608,-0.040799,-0.018241
1,12.457776,3.885604e-05,0.000019,4.416178e-06,-5.564861e-06,-6.433176e-06,0.000005,335.707062,350.870270,349.459442,...,-0.297985,-0.291245,-0.216425,-0.216418,-0.221277,-0.190081,-0.113608,-0.077773,-0.041018,-0.017465
2,12.454913,4.707007e-05,0.000035,2.491784e-06,-1.312214e-05,-4.629718e-06,-0.000006,335.804138,351.189575,350.703033,...,-0.254012,-0.271986,-0.242701,-0.249961,-0.302371,-0.286316,-0.189668,-0.085197,-0.010033,0.019464
3,13.925591,1.859126e-05,0.000031,8.427362e-06,1.655502e-06,-4.255817e-06,-0.000008,335.457703,350.354797,350.436279,...,-0.226556,-0.236710,-0.262609,-0.180884,-0.110078,-0.090459,-0.121844,-0.092352,-0.032712,0.008276
4,14.422464,1.300272e-05,0.000015,1.787337e-05,1.425643e-05,-3.775083e-06,-0.000002,335.413269,351.007812,350.952209,...,-0.184453,-0.259019,-0.333598,-0.299184,-0.241300,-0.219437,-0.191943,-0.139791,-0.071635,-0.031166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,11.359031,1.127456e-05,0.000008,-6.519937e-07,-5.312598e-07,2.857952e-06,0.000002,332.913239,349.144104,346.984772,...,-0.046995,-0.038375,-0.012014,-0.053331,-0.061202,-0.059920,-0.034129,-0.035908,-0.009429,0.001737
89,11.423564,1.215437e-05,0.000001,-3.047489e-08,-4.133225e-06,1.100732e-06,0.000004,332.137360,349.189850,346.410614,...,-0.079976,-0.073809,-0.028273,-0.057152,-0.133780,-0.192142,-0.211730,-0.145836,-0.051905,-0.009836
90,11.131371,2.257598e-05,0.000003,1.788371e-06,3.810991e-06,9.428027e-08,0.000003,332.140198,349.692963,346.497864,...,-0.161330,-0.186570,-0.187669,-0.216754,-0.237768,-0.249566,-0.214956,-0.132421,-0.039361,-0.000489
91,10.158909,8.067890e-06,0.000006,7.562204e-06,3.510273e-06,-7.782533e-06,0.000003,333.383331,349.348724,346.257874,...,-0.107938,-0.138937,-0.112077,-0.110852,-0.114616,-0.154873,-0.193884,-0.112637,-0.021895,0.014420


In [31]:
#storms 4
maindf_maha=pd.read_csv(target+'2019_w10max_nio_maha.csv',delimiter=r",")
maindf_maha.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_maha=maindf_maha.drop('a', axis=1)

extra=pd.read_csv(output+'nio/inder_2019_nio_maha.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_maha=pd.DataFrame.from_dict(TCrl_ts[4])
prlev_maha=pd.DataFrame.from_dict(TCpr_ts[4])
slev_maha=pd.DataFrame.from_dict(TCsl_ts[4])
prlev_maha['mslp']=prlev_maha['mslp']/100

ts1_maha=pd.concat([maindf_maha,extra,slev_maha,rlev_maha,prlev_maha], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[4]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[4]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[4]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[4]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_maha=pd.DataFrame.from_dict(alldict)

ts_maha=pd.concat([ts1_maha,plev_maha], axis=1, join='inner')

ts_maha

In [32]:
ts_kyaar.to_csv(output+'inner/inner_nio_all_kyaar.csv')

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,10.584517,2.837010e-05,0.000024,1.950195e-05,0.000004,-1.604122e-05,-1.840813e-05,332.681274,348.209381,347.070679,...,-0.234284,-0.305270,-0.200973,-0.006228,0.053996,0.038266,-0.086990,-0.093561,-0.052151,-0.030349
1,10.558450,2.121680e-05,0.000022,1.964651e-05,-0.000001,-1.652125e-05,-1.184978e-05,332.985962,348.588318,347.470184,...,-0.197038,-0.252120,-0.132179,0.025161,0.046073,0.030039,-0.104303,-0.093042,-0.040721,-0.016273
2,11.040154,6.358685e-06,0.000013,1.192380e-05,-0.000001,-1.260422e-05,-5.331170e-07,328.593994,349.121735,347.527954,...,-0.104746,-0.128440,-0.054533,0.030045,-0.073617,-0.154611,-0.197613,-0.124476,-0.037735,0.002339
3,10.997210,1.729269e-06,0.000016,1.408634e-05,-0.000006,-1.870668e-05,2.570313e-06,322.613495,348.714417,347.002563,...,-0.135290,-0.168127,-0.030861,0.053166,-0.053210,-0.165160,-0.174990,-0.099797,-0.023808,0.011578
4,10.445864,1.016564e-06,0.000012,1.063742e-05,-0.000002,-1.007314e-05,6.593957e-06,325.106506,348.548950,346.581116,...,-0.146544,-0.174290,-0.096480,-0.095418,-0.204810,-0.317438,-0.285635,-0.177393,-0.080935,-0.036559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,7.144583,-2.414607e-06,0.000003,1.236842e-06,0.000007,1.120639e-06,-2.032241e-07,313.049408,347.872192,344.406250,...,-0.002273,-0.014174,-0.070097,-0.057755,-0.031866,0.013215,0.003922,0.017589,0.003478,0.004849
107,7.692482,-1.646526e-06,0.000009,-3.231819e-07,-0.000003,1.121476e-06,8.999232e-07,312.210541,347.748535,344.941772,...,-0.013287,0.004856,-0.011157,-0.009142,-0.012608,0.005956,0.023715,0.025114,0.008080,0.009030
108,8.196002,2.357962e-07,0.000010,-3.733933e-06,-0.000007,-5.031806e-07,1.649358e-06,311.796631,347.566376,344.951111,...,-0.034607,-0.008843,0.002665,-0.015586,-0.019445,0.007530,0.019347,-0.006531,-0.015069,-0.011157
109,7.659556,1.868870e-06,0.000010,-6.726149e-06,-0.000005,-1.689582e-06,3.722544e-06,312.833069,347.423798,344.838867,...,-0.024693,0.002045,0.003727,-0.037727,-0.030613,0.023184,0.041601,0.003532,-0.016899,-0.021371


In [33]:
ts_maha.to_csv(output+'inner/inner_nio_all_maha.csv')

In [34]:
#storms 6
maindf_bulbul=pd.read_csv(target+'2019_w10max_nio_bulbul.csv',delimiter=r",")
maindf_bulbul.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_bulbul=maindf_bulbul.drop('a', axis=1)

extra=pd.read_csv(output+'nio/inder_2019_nio_bulbul.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_bulbul=pd.DataFrame.from_dict(TCrl_ts[5])
prlev_bulbul=pd.DataFrame.from_dict(TCpr_ts[5])
slev_bulbul=pd.DataFrame.from_dict(TCsl_ts[5])
prlev_bulbul['mslp']=prlev_bulbul['mslp']/100

ts1_bulbul=pd.concat([maindf_bulbul,extra,slev_bulbul,rlev_bulbul,prlev_bulbul], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[5]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[5]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[5]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[5]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_bulbul=pd.DataFrame.from_dict(alldict)

ts_bulbul=pd.concat([ts1_bulbul,plev_bulbul], axis=1, join='inner')

ts_bulbul

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,10.584517,0.000004,1.360982e-05,4.113630e-06,-3.951576e-06,-1.947213e-06,-5.324125e-06,332.753601,349.601562,347.492432,...,-0.049423,-0.046010,-0.030939,0.006065,0.048419,0.030083,-0.034568,-0.010628,0.011475,0.020324
1,10.558450,-0.000007,1.180217e-05,5.048701e-06,7.306909e-07,2.372099e-07,7.937452e-07,328.466949,349.497437,347.808716,...,-0.040713,-0.054359,-0.067703,-0.059106,-0.082378,-0.135567,-0.169480,-0.098193,-0.037561,-0.009307
2,11.040154,-0.000008,1.765924e-05,1.021891e-05,4.357842e-06,1.986422e-06,1.384878e-06,326.333008,349.679840,348.536407,...,-0.098930,-0.135151,-0.152608,-0.161123,-0.168720,-0.198406,-0.160609,-0.092653,-0.046288,-0.030742
3,10.997210,0.000002,1.242364e-05,-3.431449e-06,-7.650060e-06,9.642074e-07,4.334469e-06,327.476135,350.137573,349.260742,...,-0.064044,-0.031830,0.015089,-0.022348,-0.050970,-0.086196,-0.051084,-0.020695,-0.002388,0.003453
4,10.445864,0.000016,1.031823e-05,3.785833e-06,-3.954945e-06,2.207007e-06,6.988461e-06,330.158203,350.548615,348.513641,...,-0.146809,-0.145864,-0.120663,-0.177001,-0.217575,-0.192206,-0.044765,-0.005820,0.011233,0.016030
5,11.055098,0.000019,1.600657e-05,9.370015e-06,-4.528292e-07,2.002809e-06,1.070390e-06,334.466675,350.560791,348.586670,...,-0.197775,-0.211427,-0.219723,-0.244509,-0.210062,-0.132228,0.008492,0.021188,0.015150,0.012659
6,9.698686,0.000015,1.280468e-05,4.133205e-06,-2.130542e-06,-3.962862e-06,1.949821e-06,334.897827,350.389313,348.905884,...,-0.131944,-0.135610,-0.093773,-0.095354,-0.096468,-0.072053,-0.059538,-0.056230,-0.038949,-0.030409
7,10.552703,0.000025,1.823928e-05,3.255444e-06,-2.455291e-06,-3.976239e-06,3.579342e-06,335.233459,350.222290,348.732056,...,-0.204792,-0.203923,-0.150936,-0.165818,-0.166414,-0.151233,-0.149268,-0.110490,-0.052663,-0.018849
8,9.885348,0.000017,2.715001e-05,2.165952e-05,8.255443e-06,-8.395885e-06,-4.592875e-06,334.793732,350.052917,348.293823,...,-0.201297,-0.282449,-0.244417,-0.172245,-0.161043,-0.177488,-0.173951,-0.109508,-0.035493,0.001404
9,10.535303,0.000023,3.146631e-05,2.751788e-05,1.132371e-05,-7.554091e-06,-4.758706e-06,334.121826,350.374084,348.502319,...,-0.351597,-0.452209,-0.436929,-0.376704,-0.352901,-0.334757,-0.230488,-0.124491,-0.044718,-0.006083


In [35]:
ts_bulbul.to_csv(output+'inner/inner_nio_all_bulbul.csv')

In [36]:
#storms 7
maindf_pawan=pd.read_csv(target+'2019_w10max_nio_pawan.csv',delimiter=r",")
maindf_pawan.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
maindf_pawan=maindf_pawan.drop('a', axis=1)

extra=pd.read_csv(output+'nio/inder_2019_nio_pawan.csv',delimiter=r",")
extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
extra=extra.drop('a', axis=1)

rlev_pawan=pd.DataFrame.from_dict(TCrl_ts[6])
prlev_pawan=pd.DataFrame.from_dict(TCpr_ts[6])
slev_pawan=pd.DataFrame.from_dict(TCsl_ts[6])
prlev_pawan['mslp']=prlev_pawan['mslp']/100

ts1_pawan=pd.concat([maindf_pawan,extra,slev_pawan,rlev_pawan,prlev_pawan], axis=1, join='inner')

tempvortdict = {'vort_'+plev[i]:TCp_ts[6]['vort'][:,i] for i in range(21)}
temprhumdict = {'rhum_'+plev[i]:TCp_ts[6]['rhum'][:,i] for i in range(21)}
tempgpotdict = {'gpot_'+plev[i]:TCgp_ts[6]['gpot'][:,i] for i in range(21)}
tempvveldict = {'vvel_'+preslv[i]:TCvv_ts[6]['vvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict = {**dict1,**dict3}

plev_pawan=pd.DataFrame.from_dict(alldict)

ts_pawan=pd.concat([ts1_pawan,plev_pawan], axis=1, join='inner')

ts_pawan

,w10max,div100,div200,div250,div300,div400,div500,eqt1000,eqt200,eqt250,...,vvel_250,vvel_300,vvel_400,vvel_500,vvel_600,vvel_700,vvel_850,vvel_925,vvel_975,vvel_1000
0,14.006958,0.000011,0.000013,-1.946460e-07,-9.440578e-06,-2.539376e-06,8.448318e-07,334.220917,350.382416,349.181030,...,-0.101476,-0.073432,-0.030240,0.000996,-0.036418,-0.097801,-0.121072,-0.086325,-0.039240,-0.019224
1,13.472538,0.000017,0.000015,-6.533957e-06,-1.114228e-05,-3.305323e-06,2.794746e-06,332.563965,350.115356,348.919678,...,-0.110905,-0.063183,0.003510,0.008151,-0.051678,-0.102745,-0.110826,-0.081845,-0.037714,-0.015912
2,12.542070,0.000018,0.000008,-2.292947e-06,-6.008996e-06,-8.418807e-06,9.971647e-06,335.469788,349.781097,349.153442,...,-0.104973,-0.073052,0.005166,-0.023928,-0.069912,-0.043850,-0.059147,-0.057497,-0.022160,-0.001372
3,12.942327,0.000012,0.000010,3.175745e-07,1.459453e-07,-2.392968e-07,1.176803e-05,334.893250,349.381409,348.667114,...,-0.096754,-0.092295,-0.071809,-0.157434,-0.173363,-0.117155,-0.064790,-0.055217,-0.019796,0.007286
4,13.066185,0.000010,0.000023,4.677587e-06,-5.248843e-06,-1.415992e-06,9.797772e-06,328.935272,349.696594,348.902924,...,-0.199277,-0.192231,-0.111355,-0.203222,-0.199379,-0.178978,-0.149068,-0.112115,-0.050896,-0.008102
5,13.048920,0.000011,0.000016,3.257270e-06,-7.133162e-06,2.574081e-06,5.239961e-06,330.978882,350.597961,349.169556,...,-0.171095,-0.156211,-0.097280,-0.197945,-0.212733,-0.201599,-0.173338,-0.136136,-0.079711,-0.040558
6,12.942945,0.000009,0.000015,-1.096919e-06,-6.769892e-06,3.612737e-06,-2.491233e-06,334.835541,351.017822,349.089722,...,-0.155236,-0.116309,-0.095465,-0.145061,-0.149903,-0.181310,-0.307715,-0.214708,-0.088278,-0.019674
7,12.806230,0.000022,0.000010,2.474635e-06,-4.936801e-06,1.200215e-05,4.143376e-06,336.082214,351.615509,349.765442,...,-0.172518,-0.145503,-0.195666,-0.315994,-0.362486,-0.319458,-0.290083,-0.173112,-0.053119,0.011827
8,12.494928,0.000036,0.000023,5.647308e-06,-1.861424e-06,4.411397e-06,2.373149e-06,336.463989,351.797394,350.172119,...,-0.322423,-0.313914,-0.351174,-0.408257,-0.426232,-0.364190,-0.267893,-0.180641,-0.087832,-0.033876
9,13.289216,0.000052,0.000028,-4.657593e-06,-2.216604e-06,2.622066e-06,-5.397631e-06,336.556824,352.464539,350.790894,...,-0.443086,-0.405648,-0.442177,-0.447329,-0.371078,-0.279290,-0.165461,-0.128755,-0.062679,-0.016161


In [37]:
ts_pawan.to_csv(output+'inner/inner_nio_all_pawan.csv')